In [1]:
import multiprocessing
from multiprocessing import Pool
import itertools
from dla_cnn.desi.training_sets import split_sightline_into_samples 
from dla_cnn.desi.preprocess import label_sightline
from dla_cnn.spectra_utils import get_lam_data
from dla_cnn.Timer import Timer
from dla_cnn.training_set import select_samples_50p_pos_neg
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from matplotlib.pyplot import MultipleLocator
from dla_cnn.desi.DesiMock import DesiMock
from dla_cnn.desi.load_fits_files import load_DesiMocks,sightline_retriever

read_sets: Using set file -- 
  /Users/zjq/linetools/linetools/lists/sets/llist_v1.3.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
read_sets: Using set file -- 
  /Users/zjq/linetools/linetools/lists/sets/llist_v1.3.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


In [2]:
#load all fits files
files = [700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 791, 792, 793, 794, 795, 796, 797, 798, 799]
path = '/Users/zjq/7'
Mocks = load_DesiMocks(files,path)

In [3]:
#import all sightlines with DLA 17804
idlist=np.load('/Users/zjq/sightlineid_with_dla.npy',allow_pickle=True)

In [33]:
#generate 400pixel samples
data_train = {}
flux=[]
labels_classifier=[]
labels_offset=[]
col_density=[]
trainid=[]
for ii in idlist:
    sightline=sightline_retriever(ii,Mocks)
    label_sightline(sightline, kernel=400, REST_RANGE=[900,1346], pos_sample_kernel_percent=0.3)#update
    data_split=split_sightline_into_samples(sightline, REST_RANGE=[900,1346], kernel=400)#update
    sample_masks=select_samples_50p_pos_neg(sightline,kernel=400)#update
    if sample_masks.size !=0:
        trainid.append(ii)
        flux.append(np.vstack([data_split[0][m] for m in sample_masks]))
        labels_classifier.append(np.hstack([data_split[1][m] for m in sample_masks]))
        labels_offset.append(np.hstack([data_split[2][m] for m in sample_masks]))
        col_density.append(np.hstack([data_split[3][m] for m in sample_masks]))

In [34]:
#save training_data 
training_data = {trainid[i]:{'FLUX':flux[i],'labels_classifier': labels_classifier[i], 'labels_offset':labels_offset[i] , 'col_density': col_density[i]} for i in range(0,12333)}
train_save_file='/Users/zjq/training_data.npy'
np.save(train_save_file, training_data)

In [48]:
#save testing_data 
testing_data = {trainid[i]:{'FLUX':flux[i],'labels_classifier': labels_classifier[i], 'labels_offset':labels_offset[i] , 'col_density': col_density[i]} for i in range(12333,15333)}
test_save_file='/Users/zjq/testing_data.npy'
np.save(test_save_file, testing_data)